In [1]:
#https://en.wikipedia.org/wiki
import requests
from bs4 import BeautifulSoup
import re
import json
import pickle
import datetime

AttributeError: module 'html5lib.treebuilders' has no attribute '_base'

In [2]:
def name_formatter(name):
    name = re.sub("_", " ", name).strip().title()
    return(name)

In [6]:
name_formatter("Mohammad_Najibullah")

'Mohammad Najibullah'

In [3]:
def ar_lookup(name):
    """
    Convert names from English to Arabic using European Media Monitors and 
     code modified from Phil Schrodt.
    """
    name = name_formatter(name)
    base_url = "http://emm.newsexplorer.eu/NewsExplorer/search/en/entities?query="
    url = base_url + name
    
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "lxml")
        name_url = soup.find("p", {"class" : "center_headline"}).find("a")['href']
    except Exception as e:
        #print("Couldn't get page of results back: ", e)
        return []

    try:
        base = "http://emm.newsexplorer.eu/NewsExplorer/search/en/"
        name_url = base + name_url
        name_page = requests.get(name_url)
        soup = BeautifulSoup(name_page.content, "lxml")
        # check to make sure in list???? Take in alt names???/petrarch2/petrarch2/data/dictionaries
        names = soup.find("td", {"colspan" : "1"}).find_all("p")
        names = [i.text for i in names][1:]
        names_en = [i for i in names if re.search("\(.*?Eu|\(.*?en", i)]
        names_en = [re.sub("\s+?\(.+?\)", "", name) for name in names_en]
        #print("Found match. Matched English name: ", names_en[0])
        names_ar = [i for i in names if re.search("\(.*?ar", i)]
        names_ar = [re.sub("\s+?\(.+?\)", "", name) for name in names_ar]
        return names_ar
    except Exception:
        traceback.print_exc()
        return []

In [140]:
import traceback

In [157]:
#this will need the user to pass in an english name and come back with an arabic name
def hack_wiki(eng_name):
    base_url="https://en.wikipedia.org/wiki/"+eng_name
    try:
        page=requests.get(base_url)
        soup=BeautifulSoup(page.content,"lxml")
        name=soup.find(id="firstHeading").contents
        print(name)
        ar_url=soup.find("li",{"class":"interwiki-ar"}).find("a")['href']
        print(ar_url)
        ar_page=requests.get(ar_url)
        ar_soup=BeautifulSoup(ar_page.content,"lxml")
        ar_name=ar_soup.find(id="firstHeading").contents
        return {eng_name:ar_name}
        #print("name "+name+" url: "+str(ar_url))
    except Exception as e:
        #traceback.print_stack()
        print(e)

In [192]:
base_url="https://en.wikipedia.org/wiki/"+"MOHAMMAD_ZAHIR_SHAH"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
    'Content-Type': 'text/html',
}
page=requests.get(base_url,headers=headers)
soup=BeautifulSoup(page.content,"lxml")
name=soup.find(id="firstHeading").contents
print(name)
ar_url=soup.find("li",{"class":"interwiki-ar"}).find("a")['href']
print(ar_url)

['MOHAMMAD ZAHIR SHAH']


AttributeError: 'NoneType' object has no attribute 'find'

In [189]:
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>MOHAMMAD ZAHIR SHAH - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"MOHAMMAD_ZAHIR_SHAH","wgTitle":"MOHAMMAD ZAHIR SHAH","wgCurRevisionId":0,"wgRevisionId":0,"wgArticleId":0,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":[

In [175]:
hack_wiki("MOHAMMAD_ZAHIR_SHAH")

['MOHAMMAD ZAHIR SHAH']
'NoneType' object has no attribute 'find'


In [118]:
base_url="https://en.wikipedia.org/wiki/"+"Mohammad_Najibullah"
page=requests.get(base_url)
soup=BeautifulSoup(page.content,"lxml")
lists=soup.select(".infobox tr th a")

In [119]:
for item in lists:
    print(item.contents)

['President of Afghanistan']
["General Secretary of the Central Committee of the People's Democratic Party of Afghanistan"]
['Director of the State Intelligence Agency']
['Alma mater']


In [117]:
base_url="https://en.wikipedia.org/wiki/"+"Mohammad_Najibullah"
page=requests.get(base_url)
soup=BeautifulSoup(page.content,"lxml")
#lists=soup.select(".infobox tr th a")
lists=soup.find("table",{"class":"infobox"}).find_all("tr")
for item in lists:
     if(item.find("td") is not None):
        if(item.find("td").find("span",{"class":"nowrap"}) is not None):
            if(item.find("td").find("span",{"class":"nowrap"}).parent is not None):
            #if(item.find("td").find("span",{"class":"nowrap"}).find("td") is not None):
                tempstr=item.find("td").find("span",{"class":"nowrap"}).parent.contents
                strstr=str(tempstr)
                if("display:none" not in strstr):
                    print(strstr.split('\\n')[1][:-2].replace("\\xa0",''))
                
#$('.infobox tr th a').closest("tr").next("tr").find(".nowrap").closest("td").each(function(){console.log($(this).text())})

30 September 1987– 16 April 1992
4 May 1986– 16 April 1992
11 January 1980– 21 November 1985


In [120]:
def clean_line(line):
    # Take out extra space, underscores, comments, etc.
    cleaned = re.sub("_* .+", "", line).strip()
    cleaned = re.sub("_$", "", cleaned, flags=re.MULTILINE)
    return cleaned

def ingest_dictionary(dict_path):
    """
    Read in the country (or other) actor dictionaries.
    """
    with open(dict_path) as f:
        country_file = f.read()
    split_file = country_file.split("\n")
    
    dict_dict = []
    key_name = ""
    alt_names = [] 
    roles = []

    for line in split_file:
        if not line:
            pass
        elif line[0] == "#":
            pass
        elif re.match("[A-Z]", line[0]):
            # handle the previous
            entry = {"actor_en" : key_name,
                    "alt_names_en" : alt_names,
                    "roles" : roles}
            dict_dict.append(entry)
            # zero everything out
            alt_names = []
            roles = []
            # make new key name
            key_name = clean_line(line)
            # check to see if the role is built in
            if bool(re.search("\[[A-Z]{3}\]", line)):
                roles = re.findall("\[(.+?)\]", line)
        elif line[0] == "+":
            cleaned = clean_line(line[1:])
            alt_names.append(cleaned)
        elif re.match("\s", line):
            roles.append(line.strip())
    return dict_dict 
dp = "./Phoenix.Countries.actors.txt"
dict_dict = ingest_dictionary(dp)

In [122]:
len(dict_dict)

18390

In [153]:
def BuildTheTransladtionDicEngToAr(dict_dict):
    result=[]
    for dictitem in dict_dict[0:10]:
        if(dictitem["actor_en"]==''):
            continue
        try:
            result.append(hack_wiki(dictitem["actor_en"]))
        except Exception as e:
           #traceback.extract_tb(e)
            print(e)https://en.wikipedia.org/wiki/"+"Mohammad_Najibullah"
    return result

In [149]:
engToArabicDic=BuildTheTransladtionDicEngToAr(dict_dict)

  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/yan/eventData/yan-virtualenv/document_cluster/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/yan/eventData/yan-virtualenv/document_cluster/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/yan/eventData/yan-virtualenv/document_cluster/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/yan/eventData/yan-virtualenv/document_cluster/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/home/yan/eventData/yan-virtualenv/document_cluster/lib/python3.5/site-packages/tornado/ioloop.py", line 888, in start
    handler_func(fd_o

  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/yan/eventData/yan-virtualenv/document_cluster/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/yan/eventData/yan-virtualenv/document_cluster/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/yan/eventData/yan-virtualenv/document_cluster/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/yan/eventData/yan-virtualenv/document_cluster/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/home/yan/eventData/yan-virtualenv/document_cluster/lib/python3.5/site-packages/tornado/ioloop.py", line 888, in start
    handler_func(fd_o

  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/yan/eventData/yan-virtualenv/document_cluster/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/yan/eventData/yan-virtualenv/document_cluster/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/yan/eventData/yan-virtualenv/document_cluster/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/yan/eventData/yan-virtualenv/document_cluster/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/home/yan/eventData/yan-virtualenv/document_cluster/lib/python3.5/site-packages/tornado/ioloop.py", line 888, in start
    handler_func(fd_o

In [147]:
engToArabicDic

[{'AFGHANISTAN': ['أفغانستان']},
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [150]:
dict_dict[3]

{'actor_en': 'MOHAMMAD_ZAHIR_SHAH',
 'alt_names_en': [],
 'roles': ['[AFGELI 320101-331108]',
  '[AFGGOV 331108-730717]',
  '[AFGELI 730717-070723]']}

In [156]:
hack_wiki('AFGHAN_NATIONAL_SECURITY_FORCES')

'NoneType' object has no attribute 'find'


In [205]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
 
#  def init_driver():
#     driver = webdriver.Firefox()
#     driver.wait = WebDriverWait(driver, 5)
#     return driver
 
 
# def lookup(driver, query):
#     driver.get("http://www.google.com")
#     try:
#         box = driver.wait.until(EC.presence_of_element_located(
#             (By.NAME, "q")))
#         button = driver.wait.until(EC.element_to_be_clickable(
#             (By.NAME, "btnK")))
#         box.send_keys(query)
#         button.click()
#     except TimeoutException:
#         print("Box or Button not found in google.com")


In [230]:
#inorder to make the selenium work,
#we need to download geckodrive for firefox and and put it under 'usr/local/bin'
#and we need to make the driver point to where firefox instaleed.
binary = FirefoxBinary(r'/usr/bin/firefox')
caps = DesiredCapabilities.FIREFOX.copy()
#Set ‘marionette’ browser to True
caps['marionette'] = True
#Launch the Firefox instance by specifying the geckodriver executable path
driver = webdriver.Firefox(firefox_binary=binary,capabilities=caps)
driver.wait = WebDriverWait(driver, 5)

In [233]:
driver.get("https://en.wikipedia.org/wiki/"+"SIBGHATULLAH_MOJADEDI")

In [228]:
# driver.get("http://www.google.com")
# # try:
# #     box = driver.wait.until(EC.presence_of_element_located(
# #         (By.NAME, "q")))
# #     button = driver.wait.until(EC.element_to_be_clickable(
# #         (By.NAME, "btnK")))
# #     box.send_keys("Selenium")
# #     button.click()
# # except TimeoutException:
# #     print("Box or Button not found in google.com")
# # time.sleep(5)
# # driver.quit()
# search = driver.find_element_by_name('q')
# search.send_keys("google search through python")
# search.send_keys(Keys.RETURN) # hit return after you enter search text
# time.sleep(5) # sleep for 5 seconds so you can see the results
# browser.quit()

WebDriverException: Message: Expected [object Undefined] undefined to be a string


In [238]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# browser = webdriver.Firefox()
driver.get('http://www.google.com')

search = driver.find_element_by_name('q')
search.send_keys("google search through python")
search.send_keys(Keys.RETURN) # hit return after you enter search text
time.sleep(5) # sleep for 5 seconds so you can see the results
driver.quit()

WebDriverException: Message: Expected [object Undefined] undefined to be a string


In [ ]:
def init_driver():
    driver = webdriver.Firefox()
    driver.wait = WebDriverWait(driver, 5)
    return driver
 
 
def lookup(driver, query):
    driver.get("http://www.google.com")
    try:
        box = driver.wait.until(EC.presence_of_element_located(
            (By.NAME, "q")))
        button = driver.wait.until(EC.element_to_be_clickable(
            (By.NAME, "btnK")))
        box.send_keys(query)
        button.click()
    except TimeoutException:
        print("Box or Button not found in google.com")
 
 
if __name__ == "__main__":
    driver = init_driver()
    lookup(driver, "Selenium")
    time.sleep(5)
    driver.quit()